<a href="https://colab.research.google.com/github/rdwnhadi/LearningDataScience/blob/main/Transfer_Learning_(Chessman).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget --no-check-certificate \
  https://dicodingacademy.blob.core.windows.net/picodiploma/ml_pengembangan_academy/Chessman-image-dataset.zip \
  -O /tmp/Chessman-image-dataset.zip

--2021-07-15 07:11:17--  https://dicodingacademy.blob.core.windows.net/picodiploma/ml_pengembangan_academy/Chessman-image-dataset.zip
Resolving dicodingacademy.blob.core.windows.net (dicodingacademy.blob.core.windows.net)... 52.239.197.36
Connecting to dicodingacademy.blob.core.windows.net (dicodingacademy.blob.core.windows.net)|52.239.197.36|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 60684125 (58M) [application/zip]
Saving to: ‘/tmp/Chessman-image-dataset.zip’

/tmp/Chessman-image 100%[===================>]  57.87M  7.44MB/s    in 8.6s    

2021-07-15 07:11:26 (6.73 MB/s) - ‘/tmp/Chessman-image-dataset.zip’ saved [60684125/60684125]



In [2]:
import os
import zipfile
from tensorflow.keras.preprocessing.image import ImageDataGenerator
local_zip = '/tmp/Chessman-image-dataset.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()
train_dir = os.path.join('/tmp/Chessman-image-dataset/Chess')
train_datagen = ImageDataGenerator(rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    shear_range=0.2,
    fill_mode = 'nearest',
    validation_split=0.1) # set validation split

In [3]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=8,
    class_mode='categorical',
    subset='training') # set as training data
validation_generator = train_datagen.flow_from_directory(
    train_dir, # same directory as training data
    target_size=(150, 150),
    batch_size=16,
    class_mode='categorical',
    subset='validation')

Found 499 images belonging to 6 classes.
Found 52 images belonging to 6 classes.


In [4]:
import tensorflow as tf
from tensorflow.keras.layers import Input
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications import ResNet152V2
model = tf.keras.models.Sequential([
    ResNet152V2(weights="imagenet", include_top=False, input_tensor=Input(shape=(150, 150, 3))),
    # tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Flatten(), 
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(6, activation='softmax')  
])
model.layers[0].trainable = False

234553344/234545216 [==============================] - 2s 0us/step


In [5]:
model.compile(optimizer=tf.optimizers.Adam(),
              loss='categorical_crossentropy',
              metrics = ['accuracy'])

In [6]:
history = model.fit(train_generator,
                              validation_data=validation_generator,
                              epochs=50,
                              verbose=2)

Epoch 1/50
63/63 - 142s - loss: 8.9194 - accuracy: 0.4469 - val_loss: 3.6889 - val_accuracy: 0.5577
Epoch 2/50
63/63 - 130s - loss: 1.7776 - accuracy: 0.7014 - val_loss: 1.5953 - val_accuracy: 0.5962
Epoch 3/50
63/63 - 132s - loss: 1.2382 - accuracy: 0.7315 - val_loss: 0.9693 - val_accuracy: 0.6923
Epoch 4/50
63/63 - 131s - loss: 0.9216 - accuracy: 0.7816 - val_loss: 2.1269 - val_accuracy: 0.5769
Epoch 5/50
63/63 - 131s - loss: 0.6941 - accuracy: 0.8056 - val_loss: 0.8823 - val_accuracy: 0.7500
Epoch 6/50
63/63 - 130s - loss: 0.5834 - accuracy: 0.8517 - val_loss: 0.8723 - val_accuracy: 0.7115
Epoch 7/50
63/63 - 135s - loss: 0.5508 - accuracy: 0.8697 - val_loss: 0.8846 - val_accuracy: 0.8077
Epoch 8/50
63/63 - 133s - loss: 0.4181 - accuracy: 0.8758 - val_loss: 0.5249 - val_accuracy: 0.8077
Epoch 9/50
63/63 - 133s - loss: 0.4863 - accuracy: 0.8758 - val_loss: 1.0533 - val_accuracy: 0.8077
Epoch 10/50
63/63 - 132s - loss: 0.3951 - accuracy: 0.8838 - val_loss: 1.0251 - val_accuracy: 0.7692